In [2]:
import yaml
from jinja2 import Template
from itertools import product
from pathlib import Path

from evals.data import get_jsonl

##### Create all combinations of winelists and set of criteria with a cartesian product.

In [40]:
cwd = Path.cwd()

winelist_relative_path = Path("registry/data/sommelia/winelists.jsonl")
criteria_relative_path = Path("registry/data/sommelia/criteria.jsonl")

winelist_path = cwd / winelist_relative_path
criteria_path = cwd / criteria_relative_path

In [41]:
winelists = get_jsonl(winelist_path)
criteria = get_jsonl(criteria_path)

In [46]:
# Cartesian product of winelists and criteria
combinations = [{**winelist, **criterion} for winelist, criterion in product(winelists, criteria)]

##### Generate the samples by rendering the prompt template using winelist and criteria as arguements for each combination

In [ ]:
def render_template(template_id: str, winelist: str, criteria: str) -> list[dict]:
    """Renders a template with the given winelist and criteria.
    TODO: Add examples separately.
    """
    cwd = Path(__file__).parent
    with open(cwd / "templates.yaml") as f:
        templates = yaml.safe_load(f)
    template = next(t for t in templates if t["template_id"] == template_id)
    system_prompt = template["system_template"]
    user_template = Template(template["user_template"])
    user_prompt = user_template.render(winelist=winelist, criteria=criteria)

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]

    return messages


In [44]:
def write_to_jsonl(data, file_name:str):
    path = cwd / Path(f"registry/data/sommelia/{file_name}.jsonl")
    with open(path, "a") as f:
        for line in data:
            json.dump(line, f, ensure_ascii=False)
            f.write("\n")

In [45]:
write_to_jsonl(samples, "samples")